# Feature exctaction 

In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import datetime


In [3]:
df = pd.read_csv("data/basicbib.tab",  sep='\t') # main dataframe with all the patents
df["n_fcites"] = df["n_fcites"].fillna(0)

C:\Users\DIEGO UNZUETA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df_melt = pd.read_csv("data/df_424_melt.csv") #data with melted citations
df_2class = pd.read_csv("data/df_2class.csv") #

In [8]:
df_melt_all = pd.read_csv("data/df_424_melt_all.csv")

In [9]:
df_2class["issuedate"] = pd.to_datetime(df_2class["issuedate"])
df["issuedate"] = pd.to_datetime(df["issuedate"]).dt.date


## Novelty

### Herfindahl index of classes of cited patents

In [25]:

def herfindahl_index(series):
    """
    Herfindahl index of array
    """
    _, cnt = np.unique(series, return_counts=True)
    return np.square(cnt/cnt.sum()).sum()    


In [30]:
def get_herfindahl_features(dataset):
    """
    Takes in dataset, adds a herfindahl index column for the class and the subclass of cited patents
    """
    bot_index = 0
    herfindahl_class = []
    herfindahl_subclass = []
    l = dataset.shape[0]

    for p_number, (patent, bcites) in enumerate(dataset[["patent", "n_bcites"]].values):
    #     print("loop")

        top_index = bot_index + bcites

        cited = df_melt[bot_index:top_index]["cited"].values
        classes = df.loc[df["patent"].isin(cited)][["class", "subclass"]].values
        subclasses = classes[np.where(classes[:, 0].astype(int) == 424)][:, 1]

        herfindahl_class.append(herfindahl_index(classes[:, 0]))
        herfindahl_subclass.append(herfindahl_index(subclasses))

        bot_index = top_index

        if p_number % 1000 == 0:
            print(p_number, "/", l)
    dataset["herfindahl_cited_class"] = herfindahl_class
    dataset["herfindahl_cited_424subclass"] = herfindahl_subclass
    notify.send('Herfindahl Index ') 
    return(dataset)

In [1]:
df_2class = get_herfindahl_features(df_2class)

### Herfindahl index on mainline subclasses of cited patents

In [42]:
df_2class["herfindahl_cited_424subclass"]

0        0.722222
1        1.000000
2        0.142857
3        0.555556
4        0.000000
           ...   
46259    0.000000
46260    0.000000
46261    0.000000
46262    1.000000
46263    0.000000
Name: herfindahl_cited_424subclass, Length: 46264, dtype: float64

### 1.3 Number of backward citations

In [43]:
df_2class["n_bcites"]

0        14
1         1
2        10
3         3
4         0
         ..
46259     4
46260     4
46261     6
46262     3
46263     2
Name: n_bcites, Length: 46264, dtype: int64

## 2. Science-intensity

### 2.1 Number of non-patent literature references

In [36]:
df_424["n_nplcites"]

0         8
1         0
2         0
3         0
4         1
         ..
17505     0
17506     2
17507    23
17508    15
17509    63
Name: n_nplcites, Length: 17510, dtype: int64

## Growth speed

### Median age of cited patents

In [45]:
def get_median_age(dataset):
    
    """ Median age of cited patents """
    
    bot_index = 0
    medians = []
    n = dataset.shape[0]
    for p_number, (patent, bcites) in enumerate(dataset[["patent", "n_bcites"]].values):
    #     print("loop")

        top_index = bot_index + bcites #we only check the current patent cites
        cited = df_melt[bot_index:top_index]["cited"].values

        dates = df.loc[df["patent"].isin(cited)]["issuedate"].values #we grab the dates of the citations 

        if len(dates) != 0: #if we find dates of citations
            medians.append(dates[len(dates)//2]) #take the median

        else:
    #         print("zero")
            medians.append(np.nan)#otherwise we add nan 
        bot_index = top_index

        if p_number % 1000 == 0:
            print(p_number, "/", n)
            
    dataset["median_age_cited"] = medians
    
    notify.send('Median Age ') 
    return(dataset)

In [2]:
df_2class = get_median_age(df_2class)

In [55]:
# 15% missing values 
print("Missing val %: ", round(df_2class["median_age_cited"].isna().sum()/len(df_2class)*100, 3), "/100")

Missing val %:  15.606 /100


## Scope and Coverage

### Main class which patent belongs to

### Number of classes to which the patent belongs

### Number of patents registered in multiple countries with the couverage of the same invention

### Number of independent/dependent claims

In [63]:
df_424["n_claims"]

0        46.0
1         1.0
2         8.0
3        20.0
4        14.0
         ... 
17505    10.0
17506    22.0
17507    21.0
17508    18.0
17509    16.0
Name: n_claims, Length: 17510, dtype: float64

## Development effort and capabilities

### Assignee

Assignee type:

- 1: Unassigned
- 2: US Non-govt organisation
- 3: Foreign non-govt organisation
- 4: US Individual
- 5: Forgein Individual
- 6: US Federal Govt 
- 7: Foreign Govt

In [66]:
df_2class = pd.get_dummies(df_2class, columns = ["asstype"])

In [ ]:
df_2class = pd.get_dummies(df_2class, columns = ["class"])

### Number of investors

### Number of patents issued by assignee (entire dataset)

In [74]:
df_2class["issuedate"] = pd.to_datetime(df_2class["issuedate"],errors='coerce').dt.date

In [79]:
def get_countbyassignee(dataset):
    week_delta = datetime.timedelta(days=7)
    count_by_assignee = np.zeros(len(dataset))
    cap = date.fromisocalendar(1990, 1,1)

    c = 0 #index of each patent (up to len(df_424))

    # Random sanity check, more of these were performed throughout the code, most are removed 
    check = [] #counts of a random patant
    check2 = [] #indexes of a random patent

    for week in range(523):

        counts = df.loc[df["issuedate"] < cap]["assignee"].value_counts()

        df_week = dataset.loc[dataset["issuedate"] < cap].iloc[c:]

        for assignee in df_week["assignee"]:

            if dataset["assignee"][c] != assignee:
                print("ERROR: ", assignee, c)

            if (assignee in counts) & (assignee != 0):
                if assignee == 346250:
                    check.append(counts[assignee])
                    check2.append(c)          

                count_by_assignee[c] = counts[assignee]

            elif assignee == 0:
                count_by_assignee[c] = np.nan
            else:
                count_by_assignee[c] = 0

            c += 1

        if week % 50 == 0:
            print(week, "/ 521")
            
        cap += week_delta
        
    dataset["count_by_assignee_all"] = count_by_assignee
      
    notify.send('Counts by Assignee ') 
    return(dataset)

In [3]:
df_2class = get_countbyassignee(df_2class)

### Number of patents in a technology field of interest issued by an assignee

In [84]:
df_424_all = df.loc[(df["class"] == 424) | (df["class"] == "424")]
df_514_all = df.loc[(df["class"] == 514) | (df["class"] == "514")]

In [87]:
def get_patentclass(dataset, dataset_all):
    
    """
    Number of patents by assigneein class 424 or 514
    """
    
    week_delta = datetime.timedelta(days=7)
    count_by_assignee_424 = np.zeros(len(dataset))
    cap = date.fromisocalendar(1990, 1,1)

    c = 0 #index of each patent (up to len(df_424))

    for week in range(523):

        counts = dataset_all.loc[dataset_all["issuedate"] < cap]["assignee"].value_counts()

        cap += week_delta # we add the week after we take the counts so that the value of that patent is not included 

        df_week = dataset.loc[dataset["issuedate"] < cap].iloc[c:]

        for assignee in df_week["assignee"]:

            if (assignee in counts) & (assignee != 0):             
                count_by_assignee_424[c] = counts[assignee]

            elif assignee == 0:
                count_by_assignee_424[c] = np.nan
            else:
                count_by_assignee_424[c] = 0

            c += 1

        if week % 50 == 0:
            print(week, "/ 521")
            
    dataset["count_by_assignee_class"] = count_by_assignee_424
    return(dataset)


In [4]:
df_2class = get_patentclass(df_2class, df_514_all)

### 5.4 Number of patents in a technology field of interest issued by an assignee (in class 424)

In [2]:
df_2class["counts_by_assignee_other"] = df_2class["count_by_assignee_all"] - df_2class["count_by_assignee_class"]

### 5.5 Average number of forward citations of patents issued by assignee

In [93]:
def get_numcitationsall(dataset):
    week_delta = datetime.timedelta(days=7)
    inl_by_assignee_all = np.zeros(len(dataset))
    cap = date.fromisocalendar(1990, 1,1)

    c = 0 #index of each patent (up to len(df_424))

    for week in range(523):

        df_cap = df.loc[df["issuedate"] < cap]
        counts = df_cap.groupby("assignee")["n_fcites"].mean()

        df_week = dataset.loc[dataset["issuedate"] < cap].iloc[c:]

        for assignee in df_week["assignee"]:

            if (assignee in counts) & (assignee != 0):             
                inl_by_assignee_all[c] = counts[assignee]

            elif assignee == 0:
                inl_by_assignee_all[c] = np.nan
            else:
                inl_by_assignee_all[c] = 0

            c += 1

        if week % 50 == 0:
            print(week, "/ 521")

        cap += week_delta
    
    dataset["inl_by_assignee_all"] = inl_by_assignee_all
            
    notify.send('Inl by Assignee ') 
    return(dataset)

In [5]:
df_2class = get_numcitationsall(df_2class)

### Average number of forward citations of patents issued within the field (class 424) by assignee

In [98]:
def get_inlbyclass(dataset, dataset_all):
    c = 0
    week_delta = datetime.timedelta(days=7)
    inl_by_assignee_424 = []
    cap = date.fromisocalendar(1990, 1,1)
    for week in range(523):

        df_cap = dataset_all.loc[dataset_all["issuedate"] < cap]
        df_week = dataset.loc[dataset["issuedate"] < cap].iloc[c:]

        counts = df_cap.groupby("assignee")["n_fcites"].mean()

        for assignee in df_week["assignee"]:

            if (assignee in counts) & (assignee != 0):
                inl_by_assignee_424.append(counts[assignee])
            elif assignee == 0:
                inl_by_assignee_424.append(np.nan)         
            else:
                inl_by_assignee_424.append(0)

            c += 1

        if week % 50 == 0:
            print(week, "/ 521")
    #         break
        cap += week_delta
        
    dataset["inl_by_assignee_424"] = inl_by_assignee_424
    return(dataset)

In [6]:
df_2class = get_inlbyclass(df_2class, df_514_all)

### Average number of forward citations of patents issued within other fields by assignee

In [101]:
def get_inlother(dataset, pclass):
    
    df_other  =  df.loc[((df["class"].values != str(pclass)) \
                    | (df["class"].values != int(pclass))) \
                   & (df["issuedate"] > date(1990, 1,1) )\
                   & (df["issuedate"] < date(2010,1,1))]

    df_other_all  =  df.loc[((df["class"].values != str(pclass)) \
                    & (df["class"].values != int(pclass)))]

    c = 0
    week_delta = datetime.timedelta(days=7)
    inl_by_assignee_other = []
    cap = date.fromisocalendar(1990, 1,1)
    for week in range(523):

        df_cap = df_other_all.loc[df_other_all["issuedate"] < cap]
        df_week = dataset.loc[dataset["issuedate"] < cap].iloc[c:]

        counts = df_cap.groupby("assignee")["n_fcites"].mean()

        for assignee in df_week["assignee"]:

            if (assignee in counts) & (assignee != 0):
                inl_by_assignee_other.append(counts[assignee])
            elif assignee == 0:
                inl_by_assignee_other.append(np.nan)         
            else:
                inl_by_assignee_other.append(0)

            c += 1

        if week % 50 == 0:
            print(week, "/ 521")
    #         break
        cap += week_delta
        
    dataset["inl_by_assignee_other"] = inl_by_assignee_other
    return(dataset)

In [7]:
df_2class = get_inlother(df_2class, 514)